In [8]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# DATA

In [9]:
df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0)
df.head()

,string_X_train,string_Y_1,string_Y_2,Y_label,Forward,Backward,row_id,label_for_train
586,"YUNGSOX 2100M 12MT USD1,015/MT USD12,180 YUNGS...",0,13,YUNGSOX 2100M,NaN,"2MT USD1,015",855,"{'Product':[( 0 , 13 )]}"
92,"TRIS 1,2-CYCLOHEXANE DICARBOXYLIC ACID,DI-ISON...",0,50,"TRIS 1,2-CYCLOHEXANE DICARBOXYLIC ACID,DI-ISON...",CLOHEXANE DI,"OXYLIC ACID,",140,"{'Product':[( 0 , 50 )]}"
91,"TRIS . CIF PORT KLANG,MALAYSIA",0,4,TRIS,NaN,SIA,139,"{'Product':[( 0 , 4 )]}"
1000,"TRADE TERMS:CIF HEYUAN, CHINA AND/OR CIF CHANG...",73,87,PVC RESIN S-70,"NG, CHINA.1",PER MT AS PE,1425,"{'Product':[( 73 , 87 )]}"
82,TRADE TERMS INCOTERMS 2010 : CFR HAI PHONG POR...,145,166,ABS AF3535-AJ Z-BLACK,MOUNT ABS AF,-BLACK 108MT,126,"{'Product':[( 145 , 166 )]}"


# Model

In [10]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

# simple test

In [11]:
QA_input = {
    'question': 'What is your name?',
    'context': 'my name is Ricky'
}
res = nlp(QA_input)
QA_input['context'][res['start']:res['end']]

'Ricky'

# test model on this task before training

In [12]:
def test_model(n_test):
    i = 0
    table = pd.DataFrame()
    for _ in tqdm(range(n_test)):
        sample = df.sample(1)
        string_X_train = sample['string_X_train'].values[0]
        string_Y_1 = sample['string_Y_1'].values[0]
        string_Y_2 = sample['string_Y_2'].values[0]
        QA_input = {
            'question': 'What is the product name?',
            'context': string_X_train
        }
        res = nlp(QA_input)
        predict = QA_input['context'][res['start']:res['end']]
        row = pd.DataFrame({
            'label':string_X_train[string_Y_1:string_Y_2],
            'predict:':predict},index=[i])
        table = table.append(row)
        i += 1
    display(table)

In [13]:
test_model(10)

<ipython-input-12-bade752e655f>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm(range(n_test)):


,label,predict:
0,ACRYLONITRILE BUTADIENE STYRE,ACRYLONITRILE BUTADIENE STYRE
1,POLYESTER STAPLE FIBER,POLYESTER STAPLE FIBER
2,GASOIL,"FOB MAILIAO,TAIWAN"
3,HIGH IMPACT POLYSTYRENE,FORMOSA CHEMICALS AND FIBRE CORPORATION
4,SAN RESIN,CIF CHONGQING PORT
5,MONOETHYLENE GLYCOL,MONOETHYLENE GLYCOL MEG FIBER
6,POLYPROPYLENE RESIN,POLYPROPYLENE
7,PCS PVC FITTINGS,MALAYSIA INCOTERMS
8,LLDPE,SHIPPING INSTRUCTION
9,BISPHENOL-ACONTRACT,BISPHENOL
